In [1]:
# import-
%run profile
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



In [14]:
def duogame(csvfile):
    global duogroup,duo,orgduo,pubg,result
    global X,y,rg,X_train,y_train,X_test,y_test
    global killhack,wphack,DBNOshack,damagehack,assistshack,boostshack,roadkhack 
    pubg=pd.read_csv(csvfile)
    duo = pubg[pubg['matchType'].isin(['duo','duo-fpp'])]
    orgduo=duo.copy()
    # 걷지 않고 무기를 사용하는 이상치 삭제
    walk0 = duo[duo.walkDistance==0]
    wphack=walk0[walk0.weaponsAcquired>0]
    
    # 걷지 않고 킬 하는 이상치 삭제
    killhack= walk0[walk0.kills>0]
    
    # 걷지않고 도핑하는 이상치 삭제
    boostshack = walk0[walk0.boosts>0]
    
    # 걷지않고 데미지 입히는 이상치 삭제
    damagehack = walk0[walk0.damageDealt>0]
    
    # 걷지않고 어시스트 이상치 삭제
    assistshack = walk0[walk0.assists>0]
    
    # 걷지않고 기절시키는 이상치 삭제
    DBNOshack = walk0[walk0.DBNOs>0]
    
    # 차를 탑승하지 않고 로드킬하는 이상치 삭제
    ride0=duo[duo.rideDistance==0]
    roadkhack=ride0[ride0.roadKills>0]
    
    result1=pd.merge(wphack,killhack,how='outer')
    result2=pd.merge(result1,boostshack,how='outer')
    result3=pd.merge(result2,damagehack,how='outer')
    result4=pd.merge(result3,assistshack,how='outer')
    result5=pd.merge(result4,DBNOshack,how='outer')
    result=pd.merge(result5,roadkhack,how='outer')
    
    duo=duo.drop(duo[duo.Id.isin(result.Id)].index)
#     duo=duo.drop(['groupId','matchType','matchId'],axis=1).set_index('Id')
    duo=duo[['Id','walkDistance','weaponsAcquired','heals','matchDuration','boosts','killPlace','winPlacePerc']].set_index('Id')
    print("총 {}명의 사용자 중 핵사용 의심 플레이어는 {}명 이며 전체의 {:.3f}%입니다\n"
          .format(len(orgduo),len(result),len(duo)/len(orgduo)*100))
    print("이상 인원을 제외 {}팀이 플레이하였습니다\n".format(len(duo)))
    
    print("이상치 자료명은 'result'이며 정상 자료명은 'duo'입니다\n")
    
    print("핵 사용 의심 플레이어\n")
    print("이동 없이 무기소지 명단 : wphack ({}명)".format(len(wphack)))
    print("이동 없이 킬이 있는 명단 : killhack ({}명)".format(len(killhack)))
    print("이동 없이 아이템사용 명단 : boostshack ({}명)".format(len(boostshack)))
    print("이동 없이 데미지를 입힌 명단 : damagehack ({}명)".format(len(damagehack)))
    print("이동 없이 어시스트를 한 명단 : assistshack ({}명)".format(len(assistshack)))
    print("이동 없이 상대를 기절시킨 명단 : DBNOshack ({}명)".format(len(DBNOshack)))
    print("차량 탑승 없이 로드킬이 있는 명단 : roadkhack ({}명)\n".format(len(roadkhack)))
    
    print("중복인원 : {}명\n".format(len(wphack)+len(killhack)+
                              len(boostshack)+len(damagehack)+len(assistshack)+
                              len(DBNOshack)+len(roadkhack)-len(result)))
    
    X=duo.drop(['winPlacePerc'],axis=1)
    y=duo.winPlacePerc

    # RandomForestRegressor fitting & print score

    rg = RandomForestRegressor(n_estimators=100,n_jobs=-1,min_samples_leaf=1)
#     rg = RandomForestRegressor(n_jobs=-1)
    X_train,X_test,y_train,y_test = train_test_split(X,y)
    rg.fit(X_train,y_train)
    print("train data 예측률 : {:.3f}%".format(rg.score(X_train,y_train)*100))
    print("test data 예측률 : {:.3f}%".format(rg.score(X_test,y_test)*100))
    

In [15]:
duogame("train_V2.csv")

총 1310282명의 사용자 중 핵사용 의심 플레이어는 2283명 이며 전체의 99.826%입니다

이상 인원을 제외 1307999팀이 플레이하였습니다

이상치 자료명은 'result'이며 정상 자료명은 'duo'입니다

핵 사용 의심 플레이어

이동 없이 무기소지 명단 : wphack (1891명)
이동 없이 킬이 있는 명단 : killhack (27명)
이동 없이 아이템사용 명단 : boostshack (1명)
이동 없이 데미지를 입힌 명단 : damagehack (628명)
이동 없이 어시스트를 한 명단 : assistshack (64명)
이동 없이 상대를 기절시킨 명단 : DBNOshack (80명)
차량 탑승 없이 로드킬이 있는 명단 : roadkhack (35명)

중복인원 : 443명

train data 예측률 : 98.657%
test data 예측률 : 90.491%


In [241]:
# Grid Search fitting & best params 

param_grid = [{'max_features':[5,8,10],'min_samples_leaf':[1,5,10]},]
grid_search = GridSearchCV(rg,param_grid,
                           cv=5,scoring='neg_mean_squared_error')
print(grid_search.fit(X_train,y_train))
print(grid_search.best_params_)
cvres=grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'],cvres['params']):
    print(np.sqrt(-mean_score),params)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=5,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_features': [5, 8, 10], 'min_samples_leaf': [1, 5, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)
{'max_features': 10, 'min_samples_leaf': 5}
0.0655030353818135 {'max_features': 5, 'min_samples_leaf': 1}
0.06576053576184565 {'max_features': 5, 'min_samples_leaf': 5}
0.06638886983056001 {'max_features': 5, 'min_samples_leaf': 10}
0.06532910335854372 {'max_features': 8, 'min_samples_leaf': 1}
0.065351743988819

In [258]:
cvrslt_tr = cross_validate(rg,X,y,cv=5,
                           scoring=('r2','neg_mean_squared_error'),
                           return_train_score=True)
print(cvrslt_tr['train_r2'])
print(cvrslt_tr['test_r2'])

[0.99342158 0.99341228 0.99342643 0.99342152 0.99344249]
[0.953238   0.95345651 0.95283485 0.95342989 0.95340851]


In [11]:
# feature_importances_ values chart

f_importances=DataFrame(rg.feature_importances_)
f_importances.index=list(X_train.columns)
f_importances.sort_values(by=0,ascending=False)



,0
walkDistance,0.451205
killPlace,0.267273
boosts,0.081285
weaponsAcquired,0.071697
heals,0.021378
longestKill,0.016308
matchDuration,0.015228
rideDistance,0.014046
damageDealt,0.011792
numGroups,0.010266


In [248]:
rg = RandomForestRegressor(n_estimators=100,n_jobs=-1,min_samples_leaf=1,max_features=8)
X_train,X_test,y_train,y_test = train_test_split(X,y)
print(rg.fit(X_train,y_train))
print("train data 예측률 : {:.3f}%".format(rg.score(X_train,y_train)*100))
print("test data 예측률 : {:.3f}%".format(rg.score(X_test,y_test)*100))
    

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
train data 예측률 : 99.332%
test data 예측률 : 95.216%


In [263]:
duo[duo.teamKills>1]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
533,a23ca6cb11532c,c7fdf0e113ebc1,6ddc1b4b9a3d6b,0,2,123.00,0,0,2,61,...,0,0.00,0,0.000,2,0,1206.000,8,1491,0.4681
12169,63aa8fd0f1cba2,1a142d42a9fa03,1bca0be4bccdeb,0,0,0.00,1,0,2,64,...,0,0.00,0,0.000,2,0,455.800,1,0,0.3958
14451,f432c602ea6cbc,2900dde5717f81,9b818cdead9187,0,0,0.00,1,0,0,67,...,0,0.00,0,0.000,2,0,703.900,4,0,0.4082
15843,c96c9342043510,8e193dcf9ed591,0c530d369d114f,0,1,0.00,2,0,5,54,...,0,0.00,0,0.000,2,0,2391.000,2,0,0.6667
16084,5e9ce2a75c81a4,589008469cc444,796a8edef3f2ee,0,1,131.30,1,0,0,67,...,0,0.00,0,0.000,2,0,1198.000,5,1550,0.5714
16533,e2941afbff9927,e86a4547a21d05,2f0b34a55dbcfd,0,2,142.80,1,0,0,61,...,0,0.00,0,0.000,2,0,1158.000,1,1529,0.5208
19184,bbaced6de66d99,c5299f1a776faa,1a50cd101331cc,0,0,0.00,0,0,0,63,...,0,600.30,0,0.000,2,1,781.800,5,0,0.3913
19458,337e67fbfcd422,5d278e88ca278d,81a94c265fa9df,0,1,26.79,0,0,0,59,...,0,742.70,0,0.000,2,1,862.000,2,0,0.4375
20906,b0ddf1a4ad8618,986e7bf72a238c,c8dd145d8f2dfd,0,0,0.00,1,0,0,75,...,0,0.00,0,0.000,2,0,322.700,3,0,0.2083
22828,73f363943c8725,d009714f92bdf2,de259a04fff664,0,2,0.00,0,0,0,49,...,0,1230.00,0,0.000,2,1,1239.000,5,0,0.6596


In [264]:
duo.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')